# README

This is the notebook to investigate and filling in missing values.

## Filling missing values
Various techniques were used to fill the `NaN` values as much as possible.
1. `market_id`  
   984 missing values were filled in by using the latest corresponding `market_id` of the same `store_id`. There are still 3 data missing.
2. `store_primary_category`  
   154 missing values were filled in by using the latest corresponding `store_primary_category` of the same `store_id`. There are still 4606 data missing.
3. Market features - number of dasher  
  All the missing values were common across all 3 features. The data were first filled by the median from the closest hour with data of the same `store_id`, then the median from the closest hour with data of the same `market_id`. There were 120 and 12 data being filled respectivly, leaving 16130 data missing afterwards.
4. `estimated_store_to_consumer_driving_duration`  
   Unfortunately most of the data when grouped by `store_id` and `created_hours` only has 1 entry, filling with such data will be too biased.

## Removing data with no `delivery_time`
There are still 7 data with no `actual_delivery_time`, hence no target values can be obtained, the data are removed instead.

# Load libraries and declare global variables and functions

In [1]:
import pandas as pd
import eda_util
import json 

from sklearn.base import TransformerMixin


In [2]:
RANDOM_STATE = 64
TARGET = 'delivery_time'

In [3]:
def print_k_dict(d: dict, k: int) -> None:
    print({k: d[k] for k in list(d.keys())[:k]}) # Print the first k entries
    return

# Loading data

In [4]:
def load_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, index_col=0)
    dtc = eda_util.ColumnTransformers.DatatimeConverter(columns=['created_at', 'actual_delivery_time'], format='%Y-%m-%d %H:%M:%S')
    df = dtc.fit_transform(df)
    return df

In [5]:
train_csv = load_data('datasets/train.csv')
train_csv.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,created_hours,delivery_time
106702,4.0,2015-02-04 22:44:19,2015-02-04 23:16:17,6717,dessert,5.0,6,4770,5,795,795,9.0,12.0,15.0,251,407.0,22,1918.0
59877,4.0,2015-01-28 05:05:05,2015-01-28 06:11:26,2119,turkish,3.0,2,2098,2,999,1099,27.0,27.0,30.0,251,817.0,5,3981.0
88918,2.0,2015-01-24 00:51:19,2015-01-24 01:39:37,6563,steak,5.0,5,3696,4,0,1399,39.0,27.0,28.0,251,530.0,0,2898.0
112561,2.0,2015-02-18 03:29:26,2015-02-18 04:05:53,6821,sandwich,5.0,3,2147,2,235,679,99.0,64.0,108.0,251,834.0,3,2187.0
161617,1.0,2015-02-11 01:51:20,2015-02-11 03:07:53,400,japanese,1.0,1,3800,1,3800,3800,40.0,40.0,45.0,446,534.0,1,4593.0


# Data Cleaning - Missing values

In [7]:
train_info = eda_util.data_summary(train_csv)
display(train_info['na_count'])

Potential numerical features are: ['market_id', 'store_id', 'order_protocol', 'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price', 'total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders', 'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration', 'created_hours', 'delivery_time']
No potential categorical feature
Miscellenous features are: ['created_at', 'actual_delivery_time', 'store_primary_category']
Columns with missing values are: ['market_id', 'store_primary_category', 'order_protocol', 'total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders', 'estimated_store_to_consumer_driving_duration']


,na_values_count
market_id,886
store_primary_category,4284
order_protocol,913
total_onshift_dashers,14655
total_busy_dashers,14655
total_outstanding_orders,14655
estimated_store_to_consumer_driving_duration,471


## Market_id

### Fill by `store_id`

In [8]:
train_csv['market_id'].value_counts(dropna=False)

market_id
2.0    49510
4.0    42965
1.0    34172
3.0    20933
5.0    16177
6.0    13035
NaN      886
Name: count, dtype: int64

In [9]:
class MarketIDImputer(TransformerMixin):
    def __init__(self) -> None:
        self.store_to_market = {}

    def fit(self, X, y=None):
        self.store_to_market = X.sort_values(by='created_at')[['store_id', 'market_id']].dropna(subset='market_id').drop_duplicates(keep='last').set_index('store_id').to_dict()['market_id']
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X['market_id'] = X['store_id'].apply(lambda x: self.store_to_market[x] if x in self.store_to_market else None)
        return X

In [10]:
# Choose market_id by latest entry (for some reason some store corresponds to more than 1 market_id)
store_to_market = train_csv.sort_values(by='created_at')[['store_id', 'market_id']].dropna(subset='market_id').drop_duplicates(keep='last').set_index('store_id').to_dict()['market_id']
print_k_dict(store_to_market, 5)

{3174: 5.0, 5294: 3.0, 5623: 3.0, 809: 2.0, 6903: 3.0}


In [11]:
mid = MarketIDImputer()
temp_df = mid.fit_transform(train_csv)


In [13]:
# Save the store_to_market dictionary in JSON format
with open("EDA/store_to_market.json", "w") as outfile: 
    json.dump(mid.store_to_market, outfile)

In [15]:
market_id_mask = temp_df['market_id'].isna()
store_id_without_market_id = temp_df.loc[market_id_mask, 'store_id'].unique()
temp_df[temp_df['store_id'].isin(store_id_without_market_id)] # No market_id record for these four stores

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,created_hours,delivery_time
152850,NaN,2015-02-08 23:43:19,2015-02-09 00:33:20,4122,pizza,NaN,2,3400,2,1000,1400,3.0,3.0,3.0,446,586.0,23,3001.0
189608,NaN,2015-02-12 01:08:50,2015-02-12 02:08:36,6242,NaN,1.0,2,1678,2,839,839,NaN,NaN,NaN,446,1000.0,1,3586.0
120923,NaN,2015-02-08 03:39:46,2015-02-08 04:25:31,3258,NaN,NaN,3,3470,3,725,1795,NaN,NaN,NaN,446,395.0,3,2745.0
186154,NaN,2015-02-15 23:53:45,2015-02-16 00:25:42,5774,sandwich,NaN,3,1550,2,250,900,14.0,14.0,18.0,446,261.0,23,1917.0


## Store Primary Category

In [16]:
class PrimaryCategoryImputer(TransformerMixin):
    def __init__(self) -> None:
        self.store_to_primary = {}

    def fit(self, X, y=None):
        # Some of the store has more than one primary category, choosing the latest one by sorting by `created_at`
        self.store_to_primary = X.sort_values(by='created_at')[['store_id', 'store_primary_category']].dropna(subset='store_primary_category').drop_duplicates(subset='store_id', keep='last').set_index('store_id')['store_primary_category'].to_dict()
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X['store_primary_category'] = X['store_primary_category'].fillna(X['store_id'].apply(lambda x: self.store_to_primary[x] if x in self.store_to_primary else None))
        return X

In [17]:
pci = PrimaryCategoryImputer()
temp_df = pci.fit_transform(train_csv)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177678 entries, 106702 to 39366
Data columns (total 18 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   market_id                                     176792 non-null  float64       
 1   created_at                                    177678 non-null  datetime64[ns]
 2   actual_delivery_time                          177678 non-null  datetime64[ns]
 3   store_id                                      177678 non-null  int64         
 4   store_primary_category                        176791 non-null  object        
 5   order_protocol                                176765 non-null  float64       
 6   total_items                                   177678 non-null  int64         
 7   subtotal                                      177678 non-null  int64         
 8   num_distinct_items                            177678 no

In [19]:
# Save the store_to_primary dictionary in JSON format
with open("EDA/store_to_primary.json", "w") as outfile: 
    json.dump(pci.store_to_primary, outfile)

## Market Features

In [20]:
market_features = ['total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders']

In [21]:
# The missing values of the three features occur concurrently

train_csv.loc[train_csv['total_onshift_dashers'].isna(), market_features].isna().all()

total_onshift_dashers       True
total_busy_dashers          True
total_outstanding_orders    True
dtype: bool

In [22]:
class MarketFeatureImputer(TransformerMixin):
    def __init__(self, method='median', market_features=None) -> None:
        self.method = method
        self.market_features = market_features
        self.nearest = {}
        self.market_features_median = {}

    def fit(self, X, y=None):
        # Some of the store has more than one primary category, choosing the latest one by sorting by `created_at`
        if self.method == 'median':
            for feature in self.market_features:
                self.market_features_median[feature] = X.groupby(['store_id', 'created_hours'])[feature].transform('median')
        elif self.method == 'nearest':
            self.nearest = (pd.merge_asof(
                                X[['store_id', 'created_hours'] + market_features].sort_values('created_hours').reset_index(),            
                                X[['store_id', 'created_hours'] + market_features].sort_values('created_hours').dropna(subset=market_features), 
                                by='store_id',                                         
                                on='created_hours', direction='nearest'                   
                                        )
                            .set_index('index')[[feature + '_y' for feature in market_features]].rename({feature + '_y': feature for feature in market_features}, axis=1)
                            )
        else:
            raise ValueError('No such method.')
        return self

    def transform(self, X, y=None):
        X = X.copy()
        if self.method == 'median':
            for feature in self.market_features:
                X[feature] = X[feature].fillna(self.market_features_median[feature])
        elif self.method == 'nearest':
            for feature in market_features:
                X[feature] = X[feature].fillna(self.nearest[feature], downcast='infer')
        else:
            raise ValueError('No such method.')
        return X

In [23]:
mfi_median = MarketFeatureImputer(method='median', market_features=market_features)
mfi_median.fit_transform(train_csv)[market_features].isna().sum()


total_onshift_dashers       14545
total_busy_dashers          14545
total_outstanding_orders    14545
dtype: int64

In [24]:
mfi_nearest = MarketFeatureImputer(method='nearest', market_features=market_features)
mfi_nearest.fit_transform(train_csv)[market_features].isna().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_23512\323027972.py:33: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X[feature] = X[feature].fillna(self.nearest[feature], downcast='infer')


total_onshift_dashers       14535
total_busy_dashers          14535
total_outstanding_orders    14535
dtype: int64

### `market_id`?

There are a lot of missing values for `market_features` for `market_id` except 6, it maybe something to further investigate into.

In [25]:
nan_market_features = pd.DataFrame(train_csv.loc[train_csv[market_features[0]].isna(), 'market_id'].value_counts().rename('null'))
nan_market_features['non_null'] = train_csv.loc[train_csv[market_features[0]].notna(), 'market_id'].value_counts()
nan_market_features

,null,non_null
market_id,,
6.0,12445,590
3.0,1446,19487
2.0,235,49275
4.0,194,42771
1.0,187,33985
5.0,85,16092


## Prediction from other models

### estimated_store_to_consumer_driving_duration

In [26]:
train_csv.groupby(['store_id', 'created_hours'])['estimated_store_to_consumer_driving_duration'].value_counts().to_frame().value_counts()

count
1        165500
2          5105
3           400
4            53
5            14
6             1
9             1
Name: count, dtype: int64

Most of the (`store_id`, `created_hours`) groups of `estimated_store_to_consumer_driving_duration` only has 1 entry, using it to fill the missing value may be too far from actual value.

# Imputer Pipeline

In [28]:
from imputers import Imputers

In [30]:
imputers = Imputers(market_features=market_features)
imputers.pipeline

Pipeline(steps=[('market_id',
                 <imputers.MarketIDImputer object at 0x00000171639B5510>),
                ('primary_category',
                 <imputers.PrimaryCategoryImputer object at 0x000001716402D010>),
                ('market_features_median',
                 <imputers.MarketFeatureImputer object at 0x0000017169CDCE50>),
                ('market_features_nearest',
                 <imputers.MarketFeatureImputer object at 0x0000017169CDCB90>)])

In [31]:
train_csv_imputed = imputers.pipeline.fit_transform(train_csv)

d:\Data Science 101\StrataScratch Projects\Delivery Duration Prediction\doordash-delivery-prediction\imputers.py:66: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X[feature] = X[feature].fillna(self.nearest[feature], downcast='infer')


In [32]:
train_csv_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177678 entries, 106702 to 39366
Data columns (total 18 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   market_id                                     177674 non-null  float64       
 1   created_at                                    177678 non-null  datetime64[ns]
 2   actual_delivery_time                          177678 non-null  datetime64[ns]
 3   store_id                                      177678 non-null  int64         
 4   store_primary_category                        176791 non-null  object        
 5   order_protocol                                176765 non-null  float64       
 6   total_items                                   177678 non-null  int64         
 7   subtotal                                      177678 non-null  int64         
 8   num_distinct_items                            177678 no

# Saving modified data

In [33]:
# Save the filled dataset for backup

train_csv_imputed.to_csv('datasets/train_filled.csv')